# AB - Module 2

In [1]:
from scipy.stats import binom
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Задача 5
Теперь представим, что мы обзвонили 1000 клиентов и хотим проверить, с какой вероятностью 100 и меньше пользователей согласятся продлить нашу подписку.

$$p=0.3$$

In [26]:
x = np.linspace(0, 1, 100)
binom.cdf(100, 1000, 0.3)

1.7183404003317793e-52

## Задача 13

Предположим, что у нас есть 25 магазинов. Средняя выручка в день составляет 170 тысяч рублей, а оценённое по этой выборке среднеквадратичное отклонение составляет 12 тысяч рублей. Оцените 95%-й доверительный интервал средней выручки магазинов.

Для выполнения задания вам могут быть полезны функции для распределений из модуля `scipy.stats`.


In [32]:
from scipy.stats import t

In [34]:
mu = 170_000
sigma = 12_000
n = 25
alpha = 0.05

delta = t.ppf(1 - alpha / 2, df=n-1) * sigma / np.sqrt(n)

mu - delta, mu + delta

(165046.64345209274, 174953.35654790726)

## Задача 14

Давайте ещё попрактикуемся в построении доверительного интервала. В качестве выборки будем брать синтетические данные — сгенерируйте 1000 чисел из распределения Пуассона с параметром 50. Вам будет полезен модуль random в библиотеке numpy.

Посчитайте 95-й доверительный интервал среднего на основе центральной предельной теоремы. Какая ширина интервала у вас получилась? Среди вариантов ответа выбирайте ближайший к вашему.

In [6]:
data = np.random.poisson(lam=60, size=1000)

In [7]:
mu = 60
sigma = np.std(data, ddof=1)

In [8]:
n = 1000

In [9]:
delta = 1.96 * sigma / np.sqrt(n)

In [10]:
mu - delta, mu + delta

(59.52639178987617, 60.47360821012383)

## Задание 15

Воспользуемся тем же набором данных (синтетическим из распределения Пуассона).

Постройте 95-й доверительный интервал на среднеквадратичное отклонение с помощью бутстрэпа. Рекомендуется попробовать сделать это вручную (генерировать бутстрапированные выборки в цикле, сохранять нужную статистику, вычислять квантили вычисленного ряда статистик) и с помощью соответствующей функции из модуля stats библиотеки scipy. Внимание, у функции bootstrap есть параметр method, который определяет, как будут вычисляться границы доверительного интервала. В нашем случае это был percentile.

На какой интервал больше всего похож ваш?

In [11]:
from scipy.stats import bootstrap

bootstrap(
    data=(data,),
    statistic=np.std,
    confidence_level=0.95,
    n_resamples=1000,
    method='percentile',
).confidence_interval

ConfidenceInterval(low=7.29251191688019, high=7.970361806169751)

## Задача 16

Это задача предсказания средней цены в области на дом. Давайте оценим 95-процентный доверительный интервал на MSE Ridge-регрессии для этой выборки. Будем использовать бутстрэп для этого.

Для этого в цикле будем генерировать бутстрэп-выборки для обучения модели (то есть брать объекты из выборки с возвращением). В качестве тестовых объектов будем брать оставшиеся объекты. Каждый раз будем обучать модель и оценивать качество.

Реализуйте такой код для 95-процентного доверительного интервала на MSE.

Используйте 1000 итераций генераций выборки и обучения модели. Данные не обрабатывайте. Полезными методами будут `np.random.choice` для генерации индексов бутстрэп-выборок (не забудьте про «генерацию с возвращением»), `np.setdiff1d` для получения остатка выборки.

На какой интервал больше всего похож ваш результат?


In [19]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

X, y = fetch_california_housing(return_X_y=True, as_frame=True)

n = len(X)

In [22]:
n_iter = 1000

mses = []

for i in tqdm(range(n_iter)):
    train_idx = np.random.choice(list(range(n)), size=n, replace=True)
    test_idx = np.setdiff1d(list(range(n)), train_idx)

    X_test, y_test = X.values[test_idx], y.values[test_idx]
    X_train, y_train = X.values[train_idx], y.values[train_idx]

    model = Ridge()
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    mses.append(mse)

np.quantile(mses, [0.025, 0.975])


100%|██████████| 1000/1000 [00:08<00:00, 122.73it/s]


array([0.505757  , 1.03969397])

## Задача 17

Давайте оценим, насколько хорошие интервалы у нас получается строить.

Напишите функцию, которая генерирует случайную выборку из нормального распределения (давайте возьмём среднее значение, равное 3, среднеквадратичное отклонение, равное 4, а в выборке будет 1000 объектов).

Возьмите из предыдущих заданий код для построение соответствующего 95-процентного доверительного интервала для среднего на основе центральной предельной теоремы.

Повторите процедуру генерации выборки, построения доверительного интервала 1000 раз. Каждый раз проверяйте, действительно ли доверительный интервал покрыл ваше истинное среднее значение. 

Посчитайте долю случаев, когда интервал покрыл истинное значение. Ближе к какому значению вы получили ответ?

In [32]:
from numpy.random import normal

covered = []
intervals = []

for i in tqdm(range(1000)):
    data = normal(loc=3, scale=4, size=1000)

    mu = np.mean(data)
    sigma = np.std(data, ddof=0)
    delta = 1.96 * sigma / np.sqrt(1000)
    if (mu - delta < 3) and (mu + delta > 3):
        covered.append(1)
    else:
        covered.append(0)

np.mean(covered)



100%|██████████| 1000/1000 [00:00<00:00, 14745.62it/s]


0.957